# Run Inference on StarCoder2-7B

## Install and Load Dependencies

In [23]:
!pip install accelerate

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [24]:
import torch
from datetime import datetime
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

## Import Model and Tokenizer

In [25]:
checkpoint = "bigcode/starcoder2-7b"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# for fp16 use `torch_dtype=torch.float16` instead
model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto", torch_dtype=torch.float16)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## Create Test Prompt

In [39]:
prompt = """
Write a script in Python that implements a circular queue and creates one with user input. Give an example of how the script works and document the code with appropriate comments and docstrings wherever necessary. 
Generate a response only for the prompt that is given and nothing else. 
"""

## Run Inference with `model.generate()`

In [30]:
%%time
inputs = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(inputs, max_new_tokens=256)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Question: Write a script in Python that implements a circular queue and creates one with user input.
Answer:

```
class CircularQueue:
    def __init__(self, size):
        self.size = size
        self.queue = [None] * size
        self.front = 0
        self.rear = 0

    def enqueue(self, data):
        if self.is_full():
            print("Queue is full")
            return
        self.queue[self.rear] = data
        self.rear = (self.rear + 1) % self.size

    def dequeue(self):
        if self.is_empty():
            print("Queue is empty")
            return
        data = self.queue[self.front]
        self.front = (self.front + 1) % self.size
        return data

    def is_empty(self):
        return self.front == self.rear

    def is_full(self):
        return (self.rear + 1) % self.size == self.front

    def print_queue(self):
        if self.is_empty():
            print("Queue is empty")
            return
        print("Queue:")
        for i in range(self.front, sel

## Run Inference with Pipeline

In [40]:
def generateOutputWithModelPipeline(model, tokenizer, prompt, temperature=0.7, max_new_tokens=256) : 
    pipe = pipeline(
        task="text-generation", 
        model=model, 
        tokenizer=tokenizer
    )
    
    start = datetime.now()
    
    output = pipe(
        prompt,
        do_sample=True,
        max_new_tokens=max_new_tokens, 
        temperature=temperature, 
        top_k=50, 
        top_p=0.95,
        num_return_sequences=1
    )
    
    stop = datetime.now()
    
    totalTimeToPrompt = stop - start
    print(f"Execution Time : {totalTimeToPrompt}")
    
    return output

In [ ]:
output = generateOutputWithModelPipeline(model, tokenizer, prompt, max_new_tokens=1024)
print(output[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
